# Task 2: Build multi-agent workflows with Strands

In this task, you will advance from single-agent systems to sophisticated multi-agent architectures using Strands Agents' ["Agents as Tools"](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agents-as-tools/) pattern. You will create specialized financial agents that work together under an orchestrator to provide comprehensive financial guidance.

This task demonstrates enterprise-grade agent coordination, where specialized agents collaborate to solve complex problems that exceed any single agent's capabilities. You will learn hierarchical agent design, tool wrapping patterns, and intelligent request routing - skills essential for building production-scale AI systems.

![architecture](./images/multi-agent.png)


### System Architecture

Our multi-agent system consists of three core components:

#### 1. Budget Agent (from Task 1)
*Specializes in personal budgeting, spending analysis, and financial discipline*

| Tool | Description | Example Use Case |
| --- | --- | --- |
| **calculate_budget_breakdown** | 50/30/20 budget calculations for any income level | Create a budget for my $6000 monthly income |
| **analyze_spending_pattern** | Spending pattern analysis with personalized recommendations | Analyze my $800 dining expenses against $5000 income |
| **calculator** | Financial calculations and mathematical operations | Calculate 20% savings target for my budget |
  
#### 2. Financial Analysis Agent
*Focuses on investment research, portfolio management, and market analysis*

| Tool | Description | Example Use Case |
|------|-------------|------------------|
| **get_stock_analysis** | Real-time stock data and comprehensive analysis | Analyze Apple stock performance and metrics |
| **create_diversified_portfolio** | Risk-based portfolio recommendations with allocations | Create a moderate risk portfolio for $10,000 |
| **compare_stock_performance** | Multi-stock performance comparison over time periods | Compare Tesla, Apple, and Google over 6 months |

#### 3. Orchestrator Agent

*Coordinates specialized agents and synthesizes comprehensive responses*

| Capability | Description | Example Use Case |
|------------|-------------|------------------|
| **Agent Routing** | Intelligently determines which specialist(s) to consult | Routes budget questions to Budget Agent, investment queries to Financial Agent |
| **Multi-Agent Coordination** | Combines insights from multiple agents for complex queries | "Help me budget and invest" uses both agents together |
| **Response Synthesis** | Creates coherent responses from multiple agent outputs | Combines budget analysis with investment recommendations |
| **Context Management** | Maintains conversation flow across agent interactions | Remembers previous advice when making follow-up recommendations |

In [3]:
%%capture
# Install required dependencies for multi-agent system
!pip install -U -r requirements.txt --quiet --disable-pip-version-check

In [4]:
# Import dependencies for multi-agent system and stock analysis
from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager
import yfinance as yf
from typing import List
from utils import create_guardrail
import logging

# Configure logging for error tracking and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

### Task 2.1: Wrap Budget Agent as a Tool

In [6]:
# Wrap budget agent from Task 1 as a tool for orchestration
from budget_agent import FinancialReport, budget_agent

@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """Generate structured financial reports with budget analysis and recommendations."""
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        return structured_response
    except Exception as e:
        # Return a default structured response on error
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"Error generating report: {str(e)}"],
            financial_health_score=1,
        )

2026-02-18 10:08:26,648 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


### Task 2.2: Create the Financial Analysis Agent

In [8]:
# Define system prompt for investment-focused financial analysis agent
FINANCIAL_ANALYSIS_PROMPT = """You are a specialized financial analysis agent focused on investment research and portfolio recommendations. Your role is to:

1. Research and analyze stock performance data
2. Create diversified investment portfolios
3. Provide data-driven investment recommendations

You do not provide specific investment advice but rather present analytical data to help users make informed decisions. Always include disclaimers about market risks and the importance of consulting financial advisors."""


# Configure Bedrock model for financial analysis agent
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # Deterministic responses for financial advice
)

2026-02-18 10:09:20,216 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [9]:
# Tool for comprehensive stock analysis using yfinance
@tool
def get_stock_analysis(symbol: str) -> str:
    """Get comprehensive analysis for a specific stock symbol."""
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1y")

        # Calculate key metrics
        current_price = hist["Close"].iloc[-1]
        year_high = hist["High"].max()
        year_low = hist["Low"].min()
        avg_volume = hist["Volume"].mean()
        price_change = (
            (current_price - hist["Close"].iloc[0]) / hist["Close"].iloc[0]
        ) * 100

        return f"""
📊 Stock Analysis for {symbol.upper()}:
• Current Price: ${current_price:.2f}
• 52-Week High: ${year_high:.2f}
• 52-Week Low: ${year_low:.2f}
• Year-to-Date Change: {price_change:.2f}%
• Average Daily Volume: {avg_volume:,.0f} shares
• Company: {info.get("longName", "N/A")}
• Sector: {info.get("sector", "N/A")}
"""
    except Exception as e:
        return f"❌ Unable to retrieve data for {symbol}: {str(e)}"

In [11]:
# Tool for creating risk-based diversified investment portfolios
@tool
def create_diversified_portfolio(risk_level: str, investment_amount: float) -> str:
    """Create a diversified portfolio based on risk level (conservative, moderate, aggressive) and investment amount."""

    portfolios = {
        "conservative": {
            "stocks": ["JNJ", "PG", "KO", "PEP", "WMT"],
            "weights": [0.25, 0.20, 0.20, 0.20, 0.15],
            "description": "Stable, dividend-paying blue-chip stocks with low volatility",
        },
        "moderate": {
            "stocks": ["AAPL", "MSFT", "JPM", "V", "DIS"],
            "weights": [0.25, 0.25, 0.20, 0.15, 0.15],
            "description": "Mix of established tech leaders and stable financial/consumer stocks",
        },
        "aggressive": {
            "stocks": ["TSLA", "NVDA", "META", "COIN", "PLTR"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "High-growth tech and emerging sector stocks with higher volatility",
        },
    }

    if risk_level.lower() not in portfolios:
        return "❌ Risk level must be: conservative, moderate, or aggressive"

    portfolio = portfolios[risk_level.lower()]

    result = f"""
🎯 {risk_level.upper()} Portfolio Recommendation (${investment_amount:,.0f}):
{portfolio["description"]}

Portfolio Allocation:
"""

    for stock, weight in zip(portfolio["stocks"], portfolio["weights"]):
        allocation = investment_amount * weight
        result += f"• {stock}: {weight * 100:.0f}% (${allocation:,.0f})\n"

    result += "\n⚠️ Disclaimer: This is for educational purposes only. Consult a financial advisor before investing."
    return result

In [12]:
# Tool for comparing performance of multiple stocks over time periods
@tool
def compare_stock_performance(symbols: List[str], period: str = "1y") -> str:
    """Compare performance of multiple stocks over a specified period (1y, 6m, 3m, 1m)."""
    if len(symbols) > 5:
        return "❌ Please limit comparison to 5 stocks maximum"

    try:
        performance_data = {}

        for symbol in symbols:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            if not hist.empty:
                start_price = hist["Close"].iloc[0]
                end_price = hist["Close"].iloc[-1]
                performance = ((end_price - start_price) / start_price) * 100
                performance_data[symbol] = performance

        result = f"📈 Stock Performance Comparison ({period}):\n"
        sorted_stocks = sorted(
            performance_data.items(), key=lambda x: x[1], reverse=True
        )

        for stock, performance in sorted_stocks:
            result += f"• {stock}: {performance:+.2f}%\n"

        return result

    except Exception as e:
        return f"❌ Error comparing stocks: {str(e)}"

In [13]:
# Create financial analysis agent with investment tools
financial_analysis_agent = Agent(
    model=bedrock_model,  # Using the same bedrock_model from Step 1
    system_prompt=FINANCIAL_ANALYSIS_PROMPT,
    tools=[get_stock_analysis, create_diversified_portfolio, compare_stock_performance],
)

In [14]:
# Test financial analysis agent with portfolio and stock analysis
response = financial_analysis_agent(
    "Create a moderate risk portfolio for $10,000 and analyze Apple stock"
)

2026-02-18 10:10:37,206 - strands.telemetry.metrics - INFO - Creating Strands MetricsClient



Tool #1: create_diversified_portfolio

Tool #2: get_stock_analysis
Great! I've created a moderate risk portfolio for your $10,000 investment and analyzed Apple stock. Here's what I found:

## 📈 Moderate Risk Portfolio ($10,000)

Your diversified portfolio is designed to balance growth potential with stability:

| Stock | Allocation | Amount |
|-------|-----------|--------|
| **AAPL** (Apple) | 25% | $2,500 |
| **MSFT** (Microsoft) | 25% | $2,500 |
| **JPM** (JPMorgan Chase) | 20% | $2,000 |
| **V** (Visa) | 15% | $1,500 |
| **DIS** (Disney) | 15% | $1,500 |

This allocation combines:
- **Tech Leaders** (AAPL, MSFT): Growth potential from established technology companies
- **Financial Stability** (JPM): Steady dividend-paying financial institution
- **Payment Processing** (V): Resilient payment network with growth prospects
- **Entertainment** (DIS): Diversification into media and entertainment sector

---

## 📊 Apple Stock Analysis

**Current Metrics:**
- **Current Price:** $264.83
- 

In [15]:
%%writefile financial_analysis_agent.py
# Export financial analysis agent to standalone Python file

import yfinance as yf
from strands import Agent, tool
from typing import List
from strands.models import BedrockModel
import logging

# Configure logging for error tracking and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Financial Analysis Agent System Prompt
FINANCIAL_ANALYSIS_PROMPT = """You are a specialized financial analysis agent focused on investment research and portfolio recommendations. Your role is to:

1. Research and analyze stock performance data
2. Create diversified investment portfolios
3. Provide data-driven investment recommendations

You do not provide specific investment advice but rather present analytical data to help users make informed decisions. Always include disclaimers about market risks and the importance of consulting financial advisors."""

bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # Deterministic responses for financial advice
)


# Tool 1: Get Stock Analysis
@tool
def get_stock_analysis(symbol: str) -> str:
    """Get comprehensive analysis for a specific stock symbol."""
    try:
        # Fetch stock data
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1y")
        
        if hist.empty:
            return f"❌ Error: No data found for symbol '{symbol.upper()}'."
        
        # Calculate key metrics
        current_price = hist["Close"].iloc[-1]
        year_high = hist["High"].max()
        year_low = hist["Low"].min()
        avg_volume = hist["Volume"].mean()
        price_change = ((current_price - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100

        return f"""
📊 Stock Analysis for {symbol.upper()}:
• Current Price: ${current_price:.2f}
• 52-Week High: ${year_high:.2f}
• 52-Week Low: ${year_low:.2f}
• Year-to-Date Change: {price_change:.2f}%
• Average Daily Volume: {avg_volume:,.0f} shares
• Company: {info.get("longName", "N/A")}
• Sector: {info.get("sector", "N/A")}
"""
    except Exception as e:
        logger.error(f"Error retrieving data for {symbol}: {e}")
        return f"❌ Error: Unable to retrieve stock data for '{symbol}'."


# Tool 2: Create Diversified Portfolio with comprehensive error handling
@tool
def create_diversified_portfolio(risk_level: str, investment_amount: float) -> str:
    """Create a diversified portfolio based on risk level (conservative, moderate, aggressive) and investment amount."""
    try:

        # Convert to float for calculations
        amount = float(investment_amount)
        
        # Input validation - negative check
        if amount < 0:
            logger.warning(f"Negative investment amount: {amount}")
            return "❌ Error: Investment amount cannot be negative. Please provide a positive amount."
        
        # Input validation - zero check
        if amount == 0:
            return "❌ Error: Investment amount cannot be zero. Please provide a positive amount to invest."
        
        # Input validation - minimum investment
        if amount < 100:
            return "❌ Error: Investment amount too small. Please provide at least $100 for portfolio diversification."
        
        # Input validation - maximum investment
        if amount > 100_000_000:
            logger.warning(f"Unusually high investment amount: {amount}")
            return "❌ Error: Investment amount seems unusually high. Please verify the amount (maximum $100,000,000)."
        
        portfolios = {
            "conservative": {
                "stocks": ["JNJ", "PG", "KO", "PEP", "WMT"],
                "weights": [0.25, 0.20, 0.20, 0.20, 0.15],
                "description": "Stable, dividend-paying blue-chip stocks with low volatility",
            },
            "moderate": {
                "stocks": ["AAPL", "MSFT", "JPM", "V", "DIS"],
                "weights": [0.25, 0.25, 0.20, 0.15, 0.15],
                "description": "Mix of established tech leaders and stable financial/consumer stocks",
            },
            "aggressive": {
                "stocks": ["TSLA", "NVDA", "META", "COIN", "PLTR"],
                "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
                "description": "High-growth tech and emerging sector stocks with higher volatility",
            },
        }
        
        # Input validation - risk level check
        risk_level_lower = risk_level.strip().lower()
        if risk_level_lower not in portfolios:
            logger.warning(f"Invalid risk level provided: {risk_level}")
            return "❌ Error: Risk level must be 'conservative', 'moderate', or 'aggressive'. Please choose one of these options."
        
        portfolio = portfolios[risk_level_lower]
        
        result = f"""
🎯 {risk_level_lower.upper()} Portfolio Recommendation (${amount:,.0f}):
{portfolio["description"]}

Portfolio Allocation:
"""
        
        for stock, weight in zip(portfolio["stocks"], portfolio["weights"]):
            allocation = amount * weight
            result += f"• {stock}: {weight * 100:.0f}% (${allocation:,.0f})\n"
        
        result += "\n⚠️ Disclaimer: This is for educational purposes only. Consult a financial advisor before investing."
        return result
    
    except (ValueError, TypeError) as e:
        logger.error(f"Invalid input for create_diversified_portfolio: risk_level={risk_level}, amount={investment_amount}, error: {e}")
        return "❌ Error: Unable to create portfolio. Please check your inputs and try again."
    except Exception as e:
        logger.error(f"Unexpected error in create_diversified_portfolio: {e}")
        return "❌ Error: An unexpected error occurred while creating portfolio. Please try again."


# Tool 3: Compare Stock Performance with comprehensive error handling
@tool
def compare_stock_performance(symbols: List[str], period: str = "1y") -> str:
    """Compare performance of multiple stocks over a specified period (1y, 6m, 3m, 1m)."""
    try:
        # Input validation - type checking
        if not isinstance(symbols, list):
            logger.error(f"Invalid type for symbols: {type(symbols)}")
            return "❌ Error: Symbols must be provided as a list"
        
        # Input validation - empty list check
        if not symbols:
            return "❌ Error: Please provide at least one stock symbol to compare."
        
        # Input validation - list size check
        if len(symbols) > 5:
            return "❌ Error: Please limit comparison to 5 stocks maximum for better readability."
        
        # Input validation - period check
        valid_periods = ["1m", "3m", "6m", "1y", "2y", "5y"]
        if period not in valid_periods:
            logger.warning(f"Invalid period provided: {period}")
            return f"❌ Error: Invalid time period '{period}'. Please use one of: {', '.join(valid_periods)}"
        
        # Validate each symbol
        for symbol in symbols:
            if not isinstance(symbol, str):
                logger.error(f"Non-string symbol in list: {symbol}")
                return "❌ Error: All stock symbols must be text strings"
            if not symbol.strip():
                return "❌ Error: Empty stock symbol found. Please provide valid ticker symbols."
        
        performance_data = {}
        failed_symbols = []
        
        for symbol in symbols:
            symbol = symbol.strip().upper()
            try:
                stock = yf.Ticker(symbol)
                hist = stock.history(period=period)
                
                if not hist.empty and len(hist) >= 2:
                    start_price = hist["Close"].iloc[0]
                    end_price = hist["Close"].iloc[-1]
                    performance = ((end_price - start_price) / start_price) * 100
                    performance_data[symbol] = performance
                else:
                    failed_symbols.append(symbol)
            except Exception as e:
                logger.warning(f"Failed to fetch data for {symbol}: {e}")
                failed_symbols.append(symbol)
        
        # Check if we got any valid data
        if not performance_data:
            return f"❌ Error: Unable to retrieve data for any of the provided symbols. Please verify the ticker symbols are correct."
        
        result = f"📈 Stock Performance Comparison ({period}):\n"
        sorted_stocks = sorted(
            performance_data.items(), key=lambda x: x[1], reverse=True
        )
        
        for stock, performance in sorted_stocks:
            result += f"• {stock}: {performance:+.2f}%\n"
        
        # Add note about failed symbols if any
        if failed_symbols:
            result += f"\n⚠️ Note: Unable to retrieve data for: {', '.join(failed_symbols)}"
        
        return result
    
    except ValueError as e:
        logger.error(f"Value error in compare_stock_performance: {e}")
        return "❌ Error: Invalid input values. Please check your stock symbols and period."
    except Exception as e:
        logger.error(f"Unexpected error in compare_stock_performance: {e}")
        return "❌ Error: Unable to compare stock performance. Please check your internet connection and try again."


# Create the Financial Analysis Agent
financial_analysis_agent = Agent(
    model=bedrock_model,  # Using the same bedrock_model from Step 1
    system_prompt=FINANCIAL_ANALYSIS_PROMPT,
    tools=[get_stock_analysis, create_diversified_portfolio, compare_stock_performance],
    callback_handler=None,
)

if __name__ == "__main__":
    # Test the Financial Analysis Agent
    response = financial_analysis_agent(
        "Create a moderate risk portfolio for $10,000 and analyze Apple stock"
    )
    print(response)

Writing financial_analysis_agent.py


In [16]:
# Test standalone financial analysis agent
!python financial_analysis_agent.py 

Great! I've created a moderate risk portfolio for $10,000 and analyzed Apple stock. Here's what I found:

## 📈 Moderate Risk Portfolio ($10,000)

Your diversified portfolio recommendation includes:

| Stock | Allocation | Amount |
|-------|-----------|--------|
| **AAPL** (Apple) | 25% | $2,500 |
| **MSFT** (Microsoft) | 25% | $2,500 |
| **JPM** (JPMorgan Chase) | 20% | $2,000 |
| **V** (Visa) | 15% | $1,500 |
| **DIS** (Disney) | 15% | $1,500 |

This allocation balances growth potential with stability by mixing established tech leaders with stable financial and consumer stocks.

## 📊 Apple Stock Analysis

**Current Metrics:**
- **Current Price:** $264.59
- **52-Week Range:** $168.48 - $288.35
- **Year-to-Date Performance:** +8.70%
- **Average Daily Volume:** 53.96M shares
- **Sector:** Technology

Apple shows solid year-to-date performance and maintains strong trading volume, indicating good liquidity.

---

⚠️ **Important Disclaimer:** This analysis is for educational purposes only a

2026-02-18 10:11:54,533 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-18 10:11:54,689 - strands.telemetry.metrics - INFO - Creating Strands MetricsClient


### Task 2.3: Wrap Financial Analysis Agent as a Tool

In [17]:
# Wrap financial analysis agent as tool for orchestrator
from financial_analysis_agent import financial_analysis_agent


@tool
def financial_analysis_agent_tool(query: str) -> str:
    """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
    try:
        response = financial_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"❌ Financial analysis error: {str(e)}"

2026-02-18 10:17:29,343 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


### Task 2.4: Create the Orchestrator Agent

In [18]:
# Define orchestrator system prompt for multi-agent coordination
ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

In [19]:
# Create guardrail for orchestrator agent
guardrail_id, guardrail_arn = create_guardrail()


# Configure Bedrock model for orchestrator with guardrails
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # Deterministic responses for financial advice
    guardrail_id=guardrail_id,  # Your Bedrock guardrail ID
    guardrail_version="DRAFT",  # Guardrail version
    guardrail_trace="enabled",
)


# Configure conversation manager for orchestrator
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=5,  # Always keep 5 most recent messages
)


# Create orchestrator agent with both specialized agent tools
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,  # Associate a conversation manager
)

2026-02-18 10:18:53,494 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


Guardrail 'guardrail-no-bitcoin-advice' already exists. Returning existing guardrail.


In [20]:
# Test orchestrator with complex multi-agent query
response = orchestrator_agent("Compare Tesla and Apple stocks, and tell me if I can afford to invest $2000 with my $4000 monthly income.",)

I'll help you compare Tesla and Apple stocks and assess whether a $2000 investment fits your budget. Let me gather information from both my investment analysis and budgeting specialists.
Tool #1: financial_analysis_agent_tool

Tool #2: budget_agent_tool
Let me get more detailed budget analysis for you:
Tool #3: budget_agent_tool
## Comprehensive Financial Assessment

### **Stock Comparison: Tesla vs Apple**

**Tesla (TSLA)** - Higher Growth, Higher Risk
- **YTD Performance:** +17.13% (outperforming Apple)
- **Trading at:** 83% of 52-week high (strong momentum)
- **Best for:** Growth-focused investors with higher risk tolerance
- **Characteristics:** Volatile, emerging EV market leader, capital appreciation focus

**Apple (AAPL)** - Stable, Established
- **YTD Performance:** +8.86% (steady growth)
- **Trading at:** 92% of 52-week high (slight pullback)
- **Best for:** Conservative investors seeking stability
- **Characteristics:** Lower volatility, recurring services revenue, dividend-p

In [21]:
%%writefile main.py
# Export complete multi-agent system to main.py

from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager

from budget_agent import FinancialReport, budget_agent
from financial_analysis_agent import financial_analysis_agent

from utils import get_guardrail_id

ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

# Add conversation management to maintain context
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=5,  # Always keep 5 most recent messages
)

# Continue with previous configurations
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # Deterministic responses for financial advice
    guardrail_id=get_guardrail_id(),
    guardrail_version="DRAFT",
    guardrail_trace="enabled",
)


@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """Generate structured financial reports with budget analysis and recommendations."""
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        return structured_response
    except Exception as e:
        # Return a default structured response on error
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"Error generating report: {str(e)}"],
            financial_health_score=1,
        )


# Wrap Financial Analysis Agent as a Tool
@tool
def financial_analysis_agent_tool(query: str) -> str:
    """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
    try:
        response = financial_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"❌ Financial analysis error: {str(e)}"


orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,
)

if __name__ == "__main__":
    orchestrator_agent = Agent(
        model=bedrock_model,
        system_prompt=ORCHESTRATOR_PROMPT,
        tools=[budget_agent_tool, financial_analysis_agent_tool],
    )

    response = orchestrator_agent("I make $6000/month and want to start investing $500/month. Help me create a budget and suggest an investment portfolio.")

Writing main.py


In [22]:
# Test complete multi-agent orchestrator system
!python main.py 

Found guardrail 'guardrail-no-bitcoin-advice' with ID: sq4faz0bh9ku
I'll help you create a comprehensive financial plan by developing a budget and suggesting an investment portfolio. Let me gather insights from both our budgeting and investment specialists.
Tool #1: budget_agent_tool

Tool #2: financial_analysis_agent_tool
## Your Comprehensive Financial Plan

Great news! You're in a solid position to start investing while maintaining financial stability. Here's your complete plan:

### **Monthly Budget Breakdown ($6,000 income)**

| Category | Amount | % of Income |
|----------|--------|------------|
| **Housing** | $1,650 | 27.5% |
| **Food & Groceries** | $600 | 10% |
| **Transportation** | $450 | 7.5% |
| **Utilities & Phone** | $200 | 3.3% |
| **Insurance** | $300 | 5% |
| **Emergency Fund** | $600 | 10% |
| **Entertainment & Dining** | $480 | 8% |
| **Personal Care & Shopping** | $360 | 6% |
| **Debt Payments** | $500 | 8.3% |
| **Investment/Savings** | $500 | 8.3% |
| **Miscella

2026-02-18 10:22:40,029 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-18 10:22:40,316 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2026-02-18 10:22:40,921 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-18 10:22:41,052 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-18 10:22:42,831 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-18 10:22:42,946 - strands.telemetry.metrics - INFO - Creating Strands MetricsClient


**Task complete:** You have successfully built a hierarchical multi-agent system that orchestrates specialized agents for budgeting and investment analysis. Your system can now handle complex queries requiring multiple domains of expertise through intelligent routing and coordination.

### Next Steps

You have completed this notebook. To continue to the next part of the lab, return to the lab instructions and continue with **Task 3**.